# Monitoring Environmental Waste Utilization Scores
# A guide for generating EWU-Dashboards
**Sarah Schmidt & David Laner**  
*January 2023*

## 1. Configuration

### 1.1 Import packages

In [3]:
import pandas as pd
import numpy as np
import brightway2 as bw
import premise
from mycolorpy import colorlist as mcp
import string

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [4]:
# create a list with MS Excel column indices (A, B, ..., Z, AA, AB, ...)
# will be used for preparing calculations in MS Excel
alphabet = list(string.ascii_uppercase)
excel_cols=[]

for i in range(100):
    n=0
    j=i
    while j-len(alphabet)>=0:
        j=j-len(alphabet)
        n=n+1
    if n>0:
        col=alphabet[n-1]+alphabet[j]
    else:
        col=alphabet[j]
    excel_cols.append(col)

In [8]:
conversion_factors={'kg':1,
                    't':1e3,
                    'kt':1e6,
                    'g':1e-3,
                    'Mg':1e3,
                    'Gg':1e6}

### 1.2 Gather case study-specific information

In [36]:
# read various case study specific information from input data file (ExcelTool_GeneratorInput_Template.xlsx)
general_info=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='GeneralInformation', index_col=0)
general_info=general_info[general_info.columns[0]].to_dict()

Pathway_codes=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
Pathway_codes=Pathway_codes[Pathway_codes['Pathway tag'].notnull()]['Activity code'].to_list()
Pathway_names=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
Pathway_names=Pathway_names[Pathway_names['Pathway tag'].notnull()]['Pathway tag'].to_list()

WasteGen_code=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
WasteGen_code=WasteGen_code[WasteGen_code['Waste generation'].notnull()]['Activity code'].iloc[0]

act_codes=[WasteGen_code,*Pathway_codes]
act_names=[general_info['Acronym'],*Pathway_names]

# foreground system
foreground_system_codes=activity_df[activity_df['Foreground'].notnull()]['Activity code'].to_list()

# number of foreground system scenarios
n_scenarios=general_info['Maximum number of foreground system scenarios']

# materials for calculation of the environmental impact of materials
material_names=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='EnvironmentalValue')
material_names=material_names['Material Name'].to_list()
material_codes=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='EnvironmentalValue')
material_codes=material_codes['Material Code'].to_list()

waste_utilization_codes=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
waste_utilization_codes=waste_utilization_codes[pd.isnull(waste_utilization_codes['Waste utilization'])==False]['Activity code'].to_list()

# activity tags for contribution analysis
activity_tags=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
activity_tags=activity_tags['Activity tag'].to_list()
activity_tags=[tag for tag in activity_tags if pd.isnull(tag)==False]
activity_tags=list(set(activity_tags))
activity_tags.append('Others')

# waste quantity in kg
WQ_kg=general_info['Waste quantity']*conversion_factors[general_info['Unit']]

### 1.3 Setup and import of databases

#### 1.3.1 Setup

In [11]:
# create a new project or open an existing project
bw.projects.set_current(general_info['Project name'])

In [12]:
# creates the database "biosphere 3"
bw.bw2setup() 
biosphere = bw.Database("biosphere3")

Biosphere database already present!!! No setup is needed


#### 1.3.2 Ecoinvent

In [13]:
# import of the ecoinvent database
db_default_name=general_info['Database name']+'_default'
if db_default_name in bw.databases:
    print("Database has already been imported.")
    eidb_default = bw.Database(db_default_name)
else:
    # mind that the ecoinvent file must be unzipped; then: path to the datasets subfolder
    fpeidbcut = r"{}".format(general_info['Database file path'])
    # the "r" makes sure that the path is read as a string - especially useful when you have spaces in your string
    eidbcut = bw.SingleOuPathwayutEcospold2Importer(fpeidbcut, general_info['Database name']+'_default')
    eidbcut
    eidbcut.apply_strategies()
    eidbcut.statistics()
    eidb_default=eidbcut.write_database()

Database has already been imported.


In [14]:
# copy of the unmodified version of the ecoinvent database
if general_info['Database name'] in bw.databases:
    print("Database has already been imported.")
else:
    eidb_default.copy(general_info['Database name'])
eidb = bw.Database(general_info['Database name'])

Database has already been imported.


#### 1.3.3 Prospective Databases (premise)

In [15]:
# gather information which prospective scenarios shall be created
premise_scenarios=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='ProspectiveScenarios', 
                                skiprows=3, nrows=8)
premise_update=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='ProspectiveScenarios', 
                                index_col=0, skiprows=15, nrows=9, usecols='A:B')
premise_update=premise_update[premise_update['Update']=='yes']

In [17]:
# read encryption key 
# (to be requested from the premise library maintainers if you want ot use default scenarios included in `premise`)
encryption_key=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='ProspectiveScenarios', 
                         usecols = "B", header = 0, nrows=0).columns[0]

In [21]:
if general_info['Generate background system scenarios']=='yes':
    if general_info ['Type of background system scenarios']=='premise':
        
        premise_scenarios_dictlist=[]
        premise_scenario_names_list=[]

        for i in premise_scenarios.index:
            for c in premise_scenarios.columns[3:]:
                if premise_scenarios.loc[i,c]=='x':
                    scenario_dict={"model":premise_scenarios.loc[i,'IAM'], 
                                   "pathway":premise_scenarios.loc[i,'SSP']+'-'+premise_scenarios.loc[i,'RCP'], 
                                   "year":c}
                    scenario_name=premise_scenarios.loc[i,'IAM']+str(c)+'_'+premise_scenarios.loc[i,'SSP']+'-'+premise_scenarios.loc[i,'RCP']
                    premise_scenarios_dictlist.append(scenario_dict)
                    premise_scenario_names_list.append(scenario_name)
       
                    if scenario_name not in bw.databases:
                        ndb = premise.NewDatabase(
                            scenarios=[scenario_name],
                            source_db=eidb_default.name, # name of the database in the BW2 project. Must be a string.
                            source_version=general_info['Database version'], # version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
                            key=encryption_key,# <-- decryption key
                            quiet=True
                            # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
                            )


                        if len(premise_update)==8:
                            ndb.update_all()
                        else:
                            if 'Electricity' in premise_update.index:
                                ndb.update_electricity()
                            if 'Cement' in premise_update.index:
                                ndb.update_cement()
                            if 'Steel' in premise_update.index:
                                ndb.update_steel()
                            if 'Fuels' in premise_update.index:
                                ndb.update_fuels()
                            if 'Cars' in premise_update.index:
                                ndb.update_cars()
                            if 'Trucks' in premise_update.index:
                                ndb.update_trucks()
                            if 'Two wheelers' in premise_update.index:
                                ndb.update_two_wheelers()
                            if 'Buses' in premise_update.index:
                                ndb.update_buses()

                        ndb.write_db_to_brightway(name=[scenario_name])    

#### 1.3.4 Manual Background Scenarios

In [22]:
background_scenarios=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', 
                                   sheet_name='BackgroundScenarios')
background_scenarios

,background scenario,database,input,input code,input unit,input location / biosphere category,activity,activity code,activity location,amount,type


In [23]:
if general_info['Generate background system scenarios']=='yes':
    if (general_info ['Type of background system scenarios']=='manual') or (general_info ['Type of background system scenarios']=='manual + premise'):
        
        background_scenario_names_list=[]
        
        for i in background_scenarios.index:
            db_name=background_scenarios.loc[i,'database']+'_'+background_scenarios.loc[i,'background scenario']
            if db_name not in bw.databases:
                db=bw.Database(background_scenarios.loc[i,'database'])
                db.copy(db_name)
            else:
                print("Database has already been imported.")
            bsdb=bw.Database(db_name)
            
            act=bsdb.get(background_scenarios.loc[i,'activity code'])
            exc=[exc for exc in act.exchanges() if exc.input.as_dict()['code']==background_scenarios.loc[i,'input code']][0]
            
            exc['amount']=background_scenarios.loc[i,'amount']
            exc.save()
            
            if db_name not in background_scenario_names_list:
                background_scenario_names_list.append(db_name)

#### 1.3.5 Overview of databases to be included in the EWU-Dashboard

In [24]:
dbs=[eidb]

db_names_dict={}
db_names_dict['default']=eidb.name
n=1

if general_info['Generate background system scenarios']=='yes':
    if general_info ['Type of background system scenarios']=='premise':
        for scenario in premise_scenario_names_list:
            dbs.append(bw.Database(scenario))
            db='db'+str(n)
            db_names_dict[db]=scenario
            n=n+1

if general_info['Generate background system scenarios']=='yes':
    if (general_info ['Type of background system scenarios']=='manual') or (general_info ['Type of background system scenarios']=='manual + premise'):
        for scenario in background_scenario_names_list:
            dbs.append(bw.Database(scenario))
            db='db'+str(n)
            db_names_dict[db]=scenario
            n=n+1
            
db_names=[db for db in db_names_dict.keys()]

### 1.4 LCIA Methods

In [45]:
LCIA_method_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='LCIA_Methods')
LCIA_method_names=LCIA_method_df['Acronym'].to_list()

In [ ]:
LCIA_methods=[]
for i in LCIA_method_df.index:
    method=[m for m in bw.methods if m[0]=LCIA_method_df.loc[i, 'Method_Part1'] and
                                     m[1]=LCIA_method_df.loc[i, 'Method_Part2'] and
                                     m[2]=LCIA_method_df.loc[i, 'Method_Part3']][0]
    LCIA_methods.append(method)

In [47]:
bw.methods

Methods dictionary with 975 objects, including:
	('CML 2001 (superseded)', 'acidification potential', 'average European')
	('CML 2001 (superseded)', 'acidification potential', 'generic')
	('CML 2001 (superseded)', 'climate change', 'GWP 100a')
	('CML 2001 (superseded)', 'climate change', 'GWP 20a')
	('CML 2001 (superseded)', 'climate change', 'GWP 500a')
	('CML 2001 (superseded)', 'climate change', 'lower limit of net GWP')
	('CML 2001 (superseded)', 'climate change', 'upper limit of net GWP')
	('CML 2001 (superseded)', 'eutrophication potential', 'average European')
	('CML 2001 (superseded)', 'eutrophication potential', 'generic')
	('CML 2001 (superseded)', 'freshwater aquatic ecotoxicity', 'FAETP 100a')
Use `list(this object)` to get the complete list.

In [46]:
LCIA_method_names

['GW',
 'OD',
 'HTc',
 'HTnc',
 'PM',
 'IR',
 'POF',
 'AC',
 'TE',
 'FE',
 'ME',
 'ET',
 'LU',
 'RDw',
 'RDm',
 'RDf']

## 2. Calculations

### 2.1 Life Cycle Inventory

#### 2.1.1 Activities

In [25]:
activity_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')

In [26]:
for db in dbs:
    print(db.name)
    for i in activity_df.index:
        if len([act for act in db if act['code']==activity_df.loc[i,'Activity code']])==0:
            activity = db.new_activity(code = activity_df.loc[i,'Activity code'], name = activity_df.loc[i,'Activity name'], unit = activity_df.loc[i,'Unit'], location = activity_df.loc[i,'Location'])
            activity.save()
        else:
            activity=[act for act in db if act['code']==activity_df.loc[i,'Activity code']][0]
            if activity['name']!=activity_df.loc[i,'Activity name']:
                print("Error: Activity name", activity['name'], activity_df.loc[i,'Activity name'])
            if activity['location']!=activity_df.loc[i,'Location']:
                print("Error: Activity location", activity['location'], activity_df.loc[i,'Location'])
            if activity['unit']!=activity_df.loc[i,'Unit']:
                print("Error: Activity unit", activity['unit'], activity_df.loc[i,'Unit'])

ecoinvent 3.7.1_cutoff_ecoSpold02
image2030_SSP2-RCP19
image2050_SSP2-RCP19


#### 2.2.2 Exchanges

In [27]:
exchanges_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Exchanges')

In [28]:
for db in dbs:
    print(db.name)
    for actcode in exchanges_df['activity code'].unique():
        act_exchanges_df=exchanges_df[exchanges_df['activity code']==actcode]
        act=[act for act in db if act['code']==actcode][0]
        act.exchanges().delete()
        for i in act_exchanges_df.index:
            if (act_exchanges_df.loc[i,'type']=='technosphere') or (act_exchanges_df.loc[i,'type']=='production'):
                exc_input=[act for act in db if act['code']==act_exchanges_df.loc[i,'input code']][0]
            if act_exchanges_df.loc[i,'type']=='biosphere':
                exc_input=[act for act in biosphere if act['code']==act_exchanges_df.loc[i,'input code']][0]
            act.new_exchange(input = exc_input.key, amount = act_exchanges_df.loc[i,'amount'], 
                                 unit = act_exchanges_df.loc[i,'input unit'], type = act_exchanges_df.loc[i,'type']).save() 
            act.save()             

            exc=[exc for exc in act.exchanges() if exc['input']==exc_input.key][0]

            #import material flow tag
            if (exc_input['code'] in activity_df['Activity code']) & (exc_input['unit'] == 'kilogram'):
                exc['tag']='material flow'
                exc.save()
                act.save()

ecoinvent 3.7.1_cutoff_ecoSpold02
image2030_SSP2-RCP19
image2050_SSP2-RCP19


### 2.2 Modular Life Cycle Assessment

In [29]:
activity_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')

In [32]:
foreground_system=[]
for actcode in foreground_system_codes:
    foreground_system.append(eidb.get(actcode))

In [33]:
foreground_system_groups={}
for ac,actcode in enumerate(foreground_system_codes):
    foreground_system_groups[actcode]=ac

#### 2.2.1 LCI to DataFrame

In [40]:
LCI_df=pd.DataFrame()
i=0

for act in foreground_system:   
    act_name=act.as_dict()['name']
    act_code=act.as_dict()['code']
    act_unit=act.as_dict()['unit']
    act_location=act.as_dict()['location']
    #production exchange
    LCI_df.loc[i,'input']=act_name
    LCI_df.loc[i,'input code']=act_code
    LCI_df.loc[i,'input unit']=act_unit
    LCI_df.loc[i,'input location']=act_location
    LCI_df.loc[i,'activity']=act_name
    LCI_df.loc[i,'activity code']=act_code
    LCI_df.loc[i,'activity location']=act_location
    if len([exc for exc in eidb.get(act_code).production()]) >0:
        LCI_df.loc[i,'amount']=[exc.amount for exc in eidb.get(act_code).production()][0]
    else:
        LCI_df.loc[i,'amount']=1
    LCI_df.loc[i,'type']='production'
    LCI_df.loc[i,'material flow']=False
    i=i+1

    #technosphere exchanges
    for exc in act.technosphere():
        LCI_df.loc[i,'input']=exc.input.as_dict()['name']
        LCI_df.loc[i,'input code']=exc.input.as_dict()['code']
        LCI_df.loc[i,'input unit']=exc.input.as_dict()['unit']
        LCI_df.loc[i,'input location']=exc.input.as_dict()['location']
        LCI_df.loc[i,'activity']=act_name
        LCI_df.loc[i,'activity code']=act_code
        LCI_df.loc[i,'activity location']=act_location
        LCI_df.loc[i,'amount']=exc.amount
        LCI_df.loc[i,'type']='technosphere'
        if exc.input.as_dict()['code'] in foreground_system_codes:
            LCI_df.loc[i,'material flow']=True
        else:
            LCI_df.loc[i,'material flow']=False
        i=i+1
        
    #biosphere exchanges
    for exc in act.biosphere():
        LCI_df.loc[i,'input']=exc.input.as_dict()['name']
        LCI_df.loc[i,'input code']=exc.input.as_dict()['code']
        LCI_df.loc[i,'input unit']=exc.input.as_dict()['unit']
        LCI_df.loc[i,'input location']=str(exc.input.as_dict()['categories'])
        LCI_df.loc[i,'activity']=act_name
        LCI_df.loc[i,'activity code']=act_code
        LCI_df.loc[i,'activity location']=act_location
        LCI_df.loc[i,'amount']=exc.amount
        LCI_df.loc[i,'type']='biosphere'
        LCI_df.loc[i,'material flow']=False
        i=i+1

LCI_df

,input,input code,input unit,input location,activity,activity code,activity location,amount,type,material flow
0,biowaste generation,waste_generation,kilogram,DE,biowaste generation,waste_generation,DE,-1.000000e+00,production,False
1,home composting,PW1,kilogram,DE,biowaste generation,waste_generation,DE,-2.000000e-01,technosphere,True
2,industrial biowaste treatment,PW2,kilogram,DE,biowaste generation,waste_generation,DE,-8.000000e-01,technosphere,True
3,home composting,PW1,kilogram,DE,home composting,PW1,DE,-1.000000e+00,production,False
4,"home composting, apples",PW1_apples,kilogram,DE,home composting,PW1,DE,-8.000000e-01,technosphere,True
...,...,...,...,...,...,...,...,...,...,...
106,"Carbon dioxide, non-fossil",73ed05cc-9727-4abf-9516-4b5c0fe54a16,kilogram,"('air', 'urban air close to ground')","treatment of biowaste by anaerobic digestion, ...",anaerobic_digestion_bananas,DE,2.100000e-01,biosphere,False
107,Dinitrogen monoxide,6dc1b46f-ee89-4495-95c4-b8a637bcd6cb,kilogram,"('air', 'urban air close to ground')","treatment of biowaste by anaerobic digestion, ...",anaerobic_digestion_bananas,DE,3.300000e-05,biosphere,False
108,"Nitrogen, organic bound",a703733d-fabc-487b-826a-06c11ac4c0c6,kilogram,"('water',)","treatment of biowaste by anaerobic digestion, ...",anaerobic_digestion_bananas,DE,1.090000e-07,biosphere,False
109,Nitrate,5189de76-6bbb-44ba-8c42-5714f1b4371f,kilogram,"('water',)","treatment of biowaste by anaerobic digestion, ...",anaerobic_digestion_bananas,DE,2.970000e-06,biosphere,False


#### 2.2.2 Add parameters

In [34]:
# Excel sheet name (EWU-Dashboard)
params_sheet_name='Parameter'

In [38]:
scenarios=['Default']

for s in range(n_scenarios):
    scenarios.append('S'+str(s+1))

In [41]:
parameter_df=pd.DataFrame()
p=1

parameter_df.loc[0,'Parameter-ID']='P1'
parameter_df.loc[0,'Parameter']='Waste Quantity'
for s in range(n_scenarios):
    parameter_df.loc[0,'Parameter Value - S'+str(s+1)]=WQ_kg
parameter_df.loc[0,'Parameter Value - Default']=WQ_kg
parameter_df.loc[0,'Unit']='kilogram'

for i in LCI_df.index:
    if LCI_df.loc[i,'type']=='technosphere':
        param='P'+str(p+1)
        LCI_df.loc[i,'formula']=param
        parameter_df.loc[p,'Parameter-ID']=param
        parameter_df.loc[p,'Parameter']='FROM: '+LCI_df.loc[i,'input']+' ('+LCI_df.loc[i,'input location']+', '+LCI_df.loc[i,'input unit']+'), TO: '+LCI_df.loc[i,'activity']+'('+LCI_df.loc[i,'activity location']+')'
        parameter_df.loc[p,'Unit']=LCI_df.loc[i,'input unit']       
        for s in range(n_scenarios):
            parameter_df.loc[p,'Parameter Value - S'+str(s+1)]=LCI_df.loc[i,'amount']
        parameter_df.loc[p,'Parameter Value - Default']=LCI_df.loc[i,'amount']
        if LCI_df.loc[i,'material flow'] == True:
            parameter_df.loc[p, 'Group']=foreground_system_groups[LCI_df.loc[i,'activity code']]
        p=p+1
        
parameter_df

,Parameter-ID,Parameter,Parameter Value - S1,Parameter Value - S2,Parameter Value - S3,Parameter Value - S4,Parameter Value - S5,Parameter Value - Default,Unit,Group
0,P1,Waste Quantity,1.500000e+03,1.500000e+03,1.500000e+03,1.500000e+03,1.500000e+03,1.500000e+03,kilogram,NaN
1,P2,"FROM: home composting (DE, kilogram), TO: biow...",-2.000000e-01,-2.000000e-01,-2.000000e-01,-2.000000e-01,-2.000000e-01,-2.000000e-01,kilogram,0.0
2,P3,"FROM: industrial biowaste treatment (DE, kilog...",-8.000000e-01,-8.000000e-01,-8.000000e-01,-8.000000e-01,-8.000000e-01,-8.000000e-01,kilogram,0.0
3,P4,"FROM: home composting, apples (DE, kilogram), ...",-8.000000e-01,-8.000000e-01,-8.000000e-01,-8.000000e-01,-8.000000e-01,-8.000000e-01,kilogram,1.0
4,P5,"FROM: home composting, bananas (DE, kilogram),...",-2.000000e-01,-2.000000e-01,-2.000000e-01,-2.000000e-01,-2.000000e-01,-2.000000e-01,kilogram,1.0
5,P6,"FROM: treatment of biowaste, home composting, ...",-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,kilogram,2.0
6,P7,"FROM: treatment of biowaste, home composting, ...",-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,kilogram,3.0
7,P8,"FROM: industrial biowaste treatment, apples (D...",-6.000000e-01,-6.000000e-01,-6.000000e-01,-6.000000e-01,-6.000000e-01,-6.000000e-01,kilogram,4.0
8,P9,"FROM: industrial biowaste treatment, bananas (...",-4.000000e-01,-4.000000e-01,-4.000000e-01,-4.000000e-01,-4.000000e-01,-4.000000e-01,kilogram,4.0
9,P10,"FROM: treatment of biowaste, industrial compos...",-5.000000e-01,-5.000000e-01,-5.000000e-01,-5.000000e-01,-5.000000e-01,-5.000000e-01,kilogram,5.0


In [42]:
params_col_dict={}
for c,col in enumerate(parameter_df.columns):
    params_col_dict[col]=excel_cols[c]
    
params_row_dict={}
for p,param in enumerate(parameter_df['Parameter-ID']):
    params_row_dict[param]=p+2
#params_row_dict

#### 2.2.3 LCA calculations for unique exchanges

**Technosphere**

In [43]:
LCI_df_tech=LCI_df[LCI_df['type']=='technosphere']
unique_inputs_tech=LCI_df_tech['input code'].unique()
unique_activities_tech=LCI_df_tech['activity code'].unique()
CF_dict_techno={}

for d,db in enumerate(dbs):
    functional_units_tech=[]
    reference_actcodes_tech=[]

    for actcode in unique_inputs_tech:
        if actcode not in unique_activities_tech:
            reference_actcodes_tech.append(actcode)
            functional_unit={db.get(actcode):1}
            functional_units_tech.append(functional_unit)
        
    calculation_setup = {'inv': functional_units_tech, 'ia': LCIA_methods}
    bw.calculation_setups['excel tool'] = calculation_setup
    mlca = bw.MultiLCA('excel tool')
    
    CF_dict_techno_db={}

    for a,actcode in enumerate(reference_actcodes_tech):
        CF_dict_techno_flow={}
        for i,IC in enumerate(LCIA_method_names):
            CF_dict_techno_flow[IC]=mlca.results[a,i]
        CF_dict_techno_db[actcode]=CF_dict_techno_flow
    
    CF_dict_techno[db_names[d]]=CF_dict_techno_db

NameError: name 'LCIA_methods' is not defined

#### Biosphere

In [109]:
LCIA=pd.read_excel(r"C:\Users\SarahSchmidt\Desktop\5_Ind_Calc\Excel-Tool\LCIA Implementation v3.8.xlsx",sheet_name='CFs')

LCI_df_bio=LCI_df[LCI_df['type']=='biosphere']
unique_inputs_bio=LCI_df_bio['input code'].unique()
unique_activities_bio=LCI_df_bio['activity code'].unique()

functional_units_bio=[]
reference_actcodes_bio=[]

for actcode in unique_inputs_bio:
    if actcode not in unique_activities_bio:
        reference_actcodes_bio.append(actcode)
        functional_unit={biosphere.get(actcode):1}
        functional_units_bio.append(functional_unit)
        
functional_units=[*functional_units_tech,*functional_units_bio]
reference_actcodes=[*reference_actcodes_tech,*reference_actcodes_bio]

CF_dfs={}
for i,IC in enumerate(LCIA_method_names):
    CF_dfs[IC]=LCIA[(LCIA['Method']==LCIA_methods[i][0])&(LCIA['Category']==LCIA_methods[i][1])&(LCIA['Indicator']==LCIA_methods[i][2])]
    
    
CF_dict_bio={}
for actcode in reference_actcodes_bio:
    bioflow=biosphere.get(actcode)
    bioflow_name=bioflow['name']
    bioflow_c0=bioflow['categories'][0]
    if len(bioflow['categories'])==2:
        bioflow_c1=bioflow['categories'][1]
    else:
        bioflow_c1='unspecified'
    CF_dict_bio_bioflow={}
    for IC in LCIA_method_names:
        CF_df=CF_dfs[IC]
        CF_df_filtered=CF_df[(CF_df['Name']==bioflow_name)&(CF_df['Compartment']==bioflow_c0)&(CF_df['Subcompartment']==bioflow_c1)]
        if len(CF_df_filtered)==0:
            CF=0
        else:
            CF=CF_df_filtered['CF'].values[0]
        CF_dict_bio_bioflow[IC]=CF
    CF_dict_bio[actcode]=CF_dict_bio_bioflow    

#### Add characterization factors to LCI-DataFrame

In [930]:
LCI_df_sheet_names={}

for db in db_names:
    for s in scenarios:
        LCI_df_sheet_names[db+'_'+s]='LCI_'+db+'_'+s
    
LCI_df_sheet_names

{'default_Default': 'LCI_default_Default',
 'default_S1': 'LCI_default_S1',
 'default_S2': 'LCI_default_S2',
 'default_S3': 'LCI_default_S3',
 'default_S4': 'LCI_default_S4',
 'default_S5': 'LCI_default_S5',
 'db1_Default': 'LCI_db1_Default',
 'db1_S1': 'LCI_db1_S1',
 'db1_S2': 'LCI_db1_S2',
 'db1_S3': 'LCI_db1_S3',
 'db1_S4': 'LCI_db1_S4',
 'db1_S5': 'LCI_db1_S5',
 'db2_Default': 'LCI_db2_Default',
 'db2_S1': 'LCI_db2_S1',
 'db2_S2': 'LCI_db2_S2',
 'db2_S3': 'LCI_db2_S3',
 'db2_S4': 'LCI_db2_S4',
 'db2_S5': 'LCI_db2_S5'}

In [931]:
LCI_dbs={}

for d,db in enumerate(dbs):
    for s in scenarios:
        LCI_df_db=LCI_df.copy()

        for i in LCI_df_db.index:
            input_code=LCI_df_db.loc[i,'input code']
            exc_type=LCI_df_db.loc[i,'type']
            if exc_type=='biosphere':
                for IC in LCIA_method_names:
                    LCI_df_db.loc[i,IC]=CF_dict_bio[input_code][IC]
            if exc_type=='technosphere':
                if input_code in reference_actcodes_tech:
                    for IC in LCIA_method_names:
                        LCI_df_db.loc[i,IC]=CF_dict_techno[db_names[d]][input_code][IC] 
                        
        LCI_df_db['formula excel']=LCI_df_db['formula']
        parameters=parameter_df['Parameter-ID']        
            
        # Excel formula
        for i in LCI_df_db.index:
            formula=LCI_df_db.loc[i,'formula']
            if type(formula)==str:
                formula_new=formula
                for p,param in enumerate(reversed(parameters)):
                    formula_new=formula_new.replace(param,params_sheet_name+'!'\
                                +params_col_dict['Parameter Value - '+s]+str(params_row_dict[param]))
                formula_new='='+formula_new
                LCI_df_db.loc[i,'formula excel']=formula_new
            else:
                LCI_df_db.loc[i,'formula excel']='='+excel_cols[LCI_df_db.columns.get_loc('amount')]\
                                                    +str(i+2)
        
        for i in LCI_df_db.index:
            if LCI_df_db.loc[i,'type']!='production':
                for IC in LCIA_method_names:
                    LCI_df_db.loc[i,IC+' Impact']='='+excel_cols[LCI_df_db.columns.get_loc('formula excel')]\
                                                        +str(i+2)+'*'\
                                                    +excel_cols[LCI_df_db.columns.get_loc(IC)]+str(i+2)
                    
        for actcode in LCI_df_db['activity code'].unique():
            sub_df=LCI_df_db[LCI_df_db['activity code']==actcode]
            for n,IC in enumerate(LCIA_method_names):
                formula='='
                col=excel_cols[LCI_df_db.columns.get_loc(IC+' Impact')]
                for i in sub_df.index[1:]:
                    ind=str(i+2)
                    formula=formula+'+'+col+ind
                LCI_df_db.loc[sub_df.index[0], IC+' Impact']=formula
                
        LCI_df_db_prod=LCI_df_db[LCI_df_db['type']=='production']
        prod_dict={}
        
        for i in LCI_df_db_prod.index:
            actcode=LCI_df_db_prod.loc[i,'activity code']
            prod_dict[actcode]=i+2
            
        for i in LCI_df_db.index:
            if LCI_df_db.loc[i,'type']!='production':
                input_code=LCI_df_db.loc[i,'input code']
                if input_code in prod_dict.keys():
                    for n,IC in enumerate(LCIA_method_names):
                        col=excel_cols[LCI_df_db.columns.get_loc(IC+' Impact')]
                        LCI_df_db.loc[i,IC]='='+col+str(prod_dict[input_code])+'*'\
                                               +excel_cols[LCI_df_db.columns.get_loc('formula excel')]\
                                               +str(prod_dict[input_code])
                    
        LCI_dbs[db_names[d]+'_'+s]=LCI_df_db

In [932]:
LCI_df_col_dict={}
for c,col in enumerate(LCI_df_db.columns):
    LCI_df_col_dict[col]=excel_cols[c]

In [933]:
LCI_df_db

,input,input code,input unit,input location,activity,activity code,activity location,amount,type,material flow,formula,GW,OD,HTc,HTnc,PM,IR,POF,AC,TE,FE,ME,ET,LU,RDw,RDm,RDf,formula excel,GW Impact,OD Impact,HTc Impact,HTnc Impact,PM Impact,IR Impact,POF Impact,AC Impact,TE Impact,FE Impact,ME Impact,ET Impact,LU Impact,RDw Impact,RDm Impact,RDf Impact
0,biowaste generation,waste_generation,kilogram,DE,biowaste generation,waste_generation,DE,-1.000000e+00,production,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,=H2,=+AC3+AC4,=+AD3+AD4,=+AE3+AE4,=+AF3+AF4,=+AG3+AG4,=+AH3+AH4,=+AI3+AI4,=+AJ3+AJ4,=+AK3+AK4,=+AL3+AL4,=+AM3+AM4,=+AN3+AN4,=+AO3+AO4,=+AP3+AP4,=+AQ3+AQ4,=+AR3+AR4
1,home composting,TP1,kilogram,DE,biowaste generation,waste_generation,DE,-2.000000e-01,technosphere,True,P2,=AC5*AB5,=AD5*AB5,=AE5*AB5,=AF5*AB5,=AG5*AB5,=AH5*AB5,=AI5*AB5,=AJ5*AB5,=AK5*AB5,=AL5*AB5,=AM5*AB5,=AN5*AB5,=AO5*AB5,=AP5*AB5,=AQ5*AB5,=AR5*AB5,=Parameter!G3,=AB3*L3,=AB3*M3,=AB3*N3,=AB3*O3,=AB3*P3,=AB3*Q3,=AB3*R3,=AB3*S3,=AB3*T3,=AB3*U3,=AB3*V3,=AB3*W3,=AB3*X3,=AB3*Y3,=AB3*Z3,=AB3*AA3
2,industrial biowaste treatment,TP2,kilogram,DE,biowaste generation,waste_generation,DE,-8.000000e-01,technosphere,True,P3,=AC12*AB12,=AD12*AB12,=AE12*AB12,=AF12*AB12,=AG12*AB12,=AH12*AB12,=AI12*AB12,=AJ12*AB12,=AK12*AB12,=AL12*AB12,=AM12*AB12,=AN12*AB12,=AO12*AB12,=AP12*AB12,=AQ12*AB12,=AR12*AB12,=Parameter!G4,=AB4*L4,=AB4*M4,=AB4*N4,=AB4*O4,=AB4*P4,=AB4*Q4,=AB4*R4,=AB4*S4,=AB4*T4,=AB4*U4,=AB4*V4,=AB4*W4,=AB4*X4,=AB4*Y4,=AB4*Z4,=AB4*AA4
3,home composting,TP1,kilogram,DE,home composting,TP1,DE,-1.000000e+00,production,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,=H5,=+AC6+AC7,=+AD6+AD7,=+AE6+AE7,=+AF6+AF7,=+AG6+AG7,=+AH6+AH7,=+AI6+AI7,=+AJ6+AJ7,=+AK6+AK7,=+AL6+AL7,=+AM6+AM7,=+AN6+AN7,=+AO6+AO7,=+AP6+AP7,=+AQ6+AQ7,=+AR6+AR7
4,"home composting, apples",TP1_apples,kilogram,DE,home composting,TP1,DE,-8.000000e-01,technosphere,True,P4,=AC8*AB8,=AD8*AB8,=AE8*AB8,=AF8*AB8,=AG8*AB8,=AH8*AB8,=AI8*AB8,=AJ8*AB8,=AK8*AB8,=AL8*AB8,=AM8*AB8,=AN8*AB8,=AO8*AB8,=AP8*AB8,=AQ8*AB8,=AR8*AB8,=Parameter!G5,=AB6*L6,=AB6*M6,=AB6*N6,=AB6*O6,=AB6*P6,=AB6*Q6,=AB6*R6,=AB6*S6,=AB6*T6,=AB6*U6,=AB6*V6,=AB6*W6,=AB6*X6,=AB6*Y6,=AB6*Z6,=AB6*AA6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,"Carbon dioxide, non-fossil",73ed05cc-9727-4abf-9516-4b5c0fe54a16,kilogram,"('air', 'urban air close to ground')","treatment of biowaste by anaerobic digestion, bananas",anaerobic_digestion_bananas,DE,2.100000e-01,biosphere,False,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,=H108,=AB108*L108,=AB108*M108,=AB108*N108,=AB108*O108,=AB108*P108,=AB108*Q108,=AB108*R108,=AB108*S108,=AB108*T108,=AB108*U108,=AB108*V108,=AB108*W108,=AB108*X108,=AB108*Y108,=AB108*Z108,=AB108*AA108
107,Dinitrogen monoxide,6dc1b46f-ee89-4495-95c4-b8a637bcd6cb,kilogram,"('air', 'urban air close to ground')","treatment of biowaste by anaerobic digestion, bananas",anaerobic_digestion_bananas,DE,3.300000e-05,biosphere,False,NaN,298.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,=H109,=AB109*L109,=AB109*M109,=AB109*N109,=AB109*O109,=AB109*P109,=AB109*Q109,=AB109*R109,=AB109*S109,=AB109*T109,=AB109*U109,=AB109*V109,=AB109*W109,=AB109*X109,=AB109*Y109,=AB109*Z109,=AB109*AA109
108,"Nitrogen, organic bound",a703733d-fabc-487b-826a-06c11ac4c0c6,kilogram,"('water',)","treatment of biowaste by anaerobic digestion, bananas",anaerobic_digestion_bananas,DE,1.090000e-07,biosphere,False,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,=H110,=AB110*L110,=AB110*M110,=AB110*N110,=AB110*O110,=AB110*P110,=AB110*Q110,=AB110*R110,=AB110*S110,=AB110*T110,=AB110*U110,=AB110*V110,=AB110*W110,=AB110*X110,=AB110*Y110,=AB110*Z110,=AB110*AA110
109,Nitrate,5189de76-6bbb-44ba-8c42-5714f1b4371f,kilogram,"('water',)","treatment of biowa

In [934]:
LCI_df_col_dict={}
for c,col in enumerate(LCI_df_db.columns):
    LCI_df_col_dict[col]=excel_cols[c]